In [2]:
import requests

response = requests.get('http://purl.obolibrary.org/obo/go/go-basic.obo')
with open('go-basic.obo', 'w') as f:
    f.write(response.text)

In [4]:
!!pip install goatools



['Collecting goatools',
 '  Downloading goatools-1.3.11-py3-none-any.whl.metadata (14 kB)',
 'Requirement already satisfied: pandas in /opt/homebrew/anaconda3/envs/483FinalProject/lib/python3.11/site-packages (from goatools) (2.1.4)',
 'Requirement already satisfied: numpy in /opt/homebrew/anaconda3/envs/483FinalProject/lib/python3.11/site-packages (from goatools) (1.26.2)',
 'Requirement already satisfied: scipy in /opt/homebrew/anaconda3/envs/483FinalProject/lib/python3.11/site-packages (from goatools) (1.11.4)',
 'Collecting xlsxwriter (from goatools)',
 '  Downloading XlsxWriter-3.1.9-py3-none-any.whl.metadata (2.6 kB)',
 'Requirement already satisfied: statsmodels in /opt/homebrew/anaconda3/envs/483FinalProject/lib/python3.11/site-packages (from goatools) (0.14.0)',
 'Collecting openpyxl (from goatools)',
 '  Downloading openpyxl-3.1.2-py2.py3-none-any.whl (249 kB)',
 '\x1b[?25l     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.0/250.0 kB\x1b \x1b?\x1b eta \x1b-:--:--\x1b

In [17]:
from goatools.obo_parser import GODag

godag = GODag("go-basic.obo")

go-basic.obo: fmt(1.2) rel(2023-11-15) 46,228 Terms


In [25]:
adj_list = {}

for key, value in godag.items():
    if value.id not in adj_list:
        adj_list[value.id] = []
    adj_list[value.id].extend([parent.id for parent in value.parents])

# remove duplicates
for key in adj_list:
    adj_list[key] = list(set(adj_list[key]))

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [27]:
list(godag.items())[0]

('GO:0000001',
 GOTerm('GO:0000001'):
   id:GO:0000001
   item_id:GO:0000001
   name:mitochondrion inheritance
   namespace:biological_process
   _parents: 2 items
     GO:0048308
     GO:0048311
   parents: 2 items
     GO:0048308	level-05	depth-05	organelle inheritance [biological_process]
     GO:0048311	level-04	depth-06	mitochondrion distribution [biological_process]
   children: 0 items
   level:5
   depth:7
   is_obsolete:False
   alt_ids: 0 items)

In [20]:
import torch
from torch_geometric.data import Data

# Generating a mapping from GO ID to a contiguous index
node_mapping = {node: idx for idx, node in enumerate(adj_list.keys())}

# Recreate edges_list using the new mapping
edges_list = [(node_mapping[key], node_mapping[parent]) for key in adj_list for parent in adj_list[key]]

# Now you can convert edges_list to tensor
edge_index = torch.tensor(edges_list, dtype=torch.long).t().contiguous()

# Define PyG Data
data = Data(edge_index=edge_index)

In [24]:
from matplotlib import pyplot as plt
import networkx as nx

G = nx.from_edgelist(data.edge_index.t().tolist())
G.nodes

NodeView((0, 22779, 22781, 1, 4657, 2, 5242, 3, 3515, 4, 5, 22, 6, 3025, 7, 4678, 8, 4124, 9, 2915, 10, 37762, 11, 2936, 12, 18467, 13, 24300, 14, 15, 24453, 38237, 16, 3880, 17, 21021, 18, 21022, 19, 20, 11550, 21, 9074, 23, 24, 12812, 3951, 25, 26, 9117, 10334, 27, 3541, 28, 19772, 34695, 29, 4379, 30, 11638, 31, 28473, 39977, 32, 34499, 34496, 9071, 33, 2405, 34, 19004, 10696, 36938, 35, 36969, 36, 29786, 4330, 37, 12809, 14834, 24808, 24411, 38, 39, 40, 36939, 34319, 41, 7962, 42, 452, 43, 34062, 44, 16473, 37323, 45, 12870, 46, 18317, 47, 39066, 29355, 48, 24526, 49, 19883, 19938, 50, 24528, 51, 24527, 52, 19934, 53, 32157, 178, 54, 24532, 55, 24531, 56, 24530, 57, 58, 59, 24534, 60, 37029, 7892, 61, 36936, 4513, 10797, 62, 21062, 19505, 63, 19506, 36937, 21063, 64, 7957, 65, 29785, 66, 29784, 4565, 67, 37027, 3460, 7973, 68, 69, 8974, 70, 5589, 4303, 71, 9079, 72, 34776, 73, 74, 75, 76, 77, 78, 3645, 79, 9100, 80, 20619, 4165, 81, 12598, 82, 40269, 28046, 83, 31052, 84, 85, 32020